* Download the latest release and unzip it. Make a new directory under the user directory an copy all the unzipped folders and files into it.
* Open the log file log4j.properties.template. Change INFO to WARN,  save and close the file. 
* In the same directory remove .template from every filename.
* Go to Control Panel -> System and Security -> System -> Advanced Settings -> Environment Variables and create a new user variable named SPARK_HOME. Fill the value-field with the directory to Spark, e.g. C:\Users\XXXXX\Spark.
* Add %SPARK_HOME%\bin to the path variable.
* Download winutils, extract the content to c:\winutils\bin
* Add HADOOP_HOME  -> C:\winutils and %HADOOP_HOME%\bin to the user/system variables
* Make sure you have installed java
* Open command line and run spark-shell. If that does not work: change into C:\Users\XXXX\Spark\bin and try again.

![image.png](Spark.JPG)

##### Leider kriege ich es auf dem notebook nicht mehr zum Laufen, deswegen habe ich in VSC weitergemacht... im Mai lief es hier noch.

import findspark
findspark.init()
from pyspark import SparkConf, SparkContext
from operator import add
import re
from English_Stop_Words import english_stop_words

conf = SparkConf()
conf.set("spark.driver.host", "localhost")
conf.setMaster('local')
conf.setAppName('Spark Basics Exercise')
sc = SparkContext(conf=conf)

##### Einlesen des vorgesäuberten Textfiles ins RDD
rdd_text = sc.textFile("t8.shakespeare_cleaned.txt")

##### Textbereinigung und Zählen der verschiedenen Wörter:
wordcount = rdd_text \
    .flatMap(lambda line: line.split(' ')) \
    .map(lambda word: word.lower()) \
    .map(lambda word: re.sub(r'\'+[a-z]', '', word)) \
    .map(lambda word: re.sub(r'[\n\b\f\r\t\v,.!?:;\"[\]{}()&\-\'\x00]', ' ', word)) \
    .map(lambda word: word.replace('\"', '')) \
    .map(lambda word: word.strip()) \
    .map(lambda word: re.sub(r'\s+[a-z]+$', '', word)) \
    .map(lambda word: re.sub(r'[s]$', '', word)) \
    .map(lambda word: (word, 1)) \
    .reduceByKey(add)

print("Anzahl der Wörter: " + str(wordcount.count()))

##### Wir erstellen eine zweite Liste, bei der wir zum Vergleich die sog. stop-words und alle Wörter, die aus weniger als 3 Zeichen bestehen herausfiltern:
filtered_wordcount = wordcount.filter(lambda x: len(x[0]) > 2 and not x[0] in english_stop_words)
print("Anzahl der Wörter ohne Stop-Wörter: " + str(filtered_wordcount.count()))

##### Wir erstellen 2 Listen:
most_frequent_words_filtered = filtered_wordcount.takeOrdered(24, key=lambda x: -x[1])
wordcount = wordcount.filter(lambda x: len(x[0]) > 0)
most_frequent_words = wordcount.takeOrdered(24, key=lambda x: -x[1])

##### Wir erstellen 2 Textfiles, einmal die ungefilterte und einmal die gefilterte Liste
with open('most_frequent_words.txt', 'w') as filehandle:
    line_number = 1
    for count_item in most_frequent_words:
        filehandle.write(str(line_number) + "\t" + '%s\t\t-\tOccurrences: %d\n' % (count_item[0], count_item[1]))
        line_number = line_number +1

with open('most_frequent_words_filtered.txt', 'w') as filehandle:
    line_number = 1
    for count_item in most_frequent_words_filtered:
        filehandle.write(str(line_number) + "\t" + '%s\t\t-\tOccurrences: %d\n' % (count_item[0], count_item[1]))
        line_number = line_number +1

Output:
Anzahl der Wörter: 20850
Anzahl der Wörter ohne Stop-Wörter: 20356

Ungefiltert:

*  1	i		-	Occurrences: 31413
*  2	the		-	Occurrences: 27319
*  3	and		-	Occurrences: 26023
*  4	a		-	Occurrences: 20203
*  5	to		-	Occurrences: 19736
*  6	of		-	Occurrences: 17471
*  7	you		-	Occurrences: 13631
*  8	my		-	Occurrences: 12464
*  9	that	-	Occurrences: 11281
* 10	in		-	Occurrences: 11071
* 11	not		-	Occurrences: 8499
* 12	with	-	Occurrences: 7779
* 13	me		-	Occurrences: 7778
* 14	it		-	Occurrences: 7722
* 15	for		-	Occurrences: 7624
* 16	your	-	Occurrences: 6901
* 17	be		-	Occurrences: 6870
* 18	hi		-	Occurrences: 6869
* 19	he		-	Occurrences: 6600
* 20	thi		-	Occurrences: 6589
* 21	but		-	Occurrences: 6256
* 22	have	-	Occurrences: 5888
* 23	thou	-	Occurrences: 5485
* 24	him		-	Occurrences: 5204

Gefiltert

*  1	thi		-	Occurrences: 6589
*  2	thou	-	Occurrences: 5485
*  3	thy		-	Occurrences: 4031
*  4	shall	-	Occurrences: 3591
*  5	lord	-	Occurrences: 3555
*  6	thee	-	Occurrences: 3181
*  7	king	-	Occurrences: 3155
*  8	come	-	Occurrences: 3125
*  9	good	-	Occurrences: 2835
* 10	sir		-	Occurrences: 2803
* 11	let		-	Occurrences: 2371
* 12	love	-	Occurrences: 2371
* 13	enter	-	Occurrences: 2122
* 14	man		-	Occurrences: 2004
* 15	make	-	Occurrences: 1992
* 16	hath	-	Occurrences: 1939
* 17	say		-	Occurrences: 1925
* 18	know	-	Occurrences: 1880
* 19	like	-	Occurrences: 1729
* 20	did		-	Occurrences: 1626
* 21	speak	-	Occurrences: 1297
* 22	time	-	Occurrences: 1287
* 23	heart	-	Occurrences: 1238
* 24	god		-	Occurrences: 1184
